# 模型设计

本次采用具有两个隐藏层的卷积神经网络，需要注意的是，对于torch.nn.conv2d这个模型来说，它需要的输入是4维的，因为初始化时它的第一个卷积层是(ouput_channle, dimension, kenral_size, kernal_size)，所以输入的形式应当是(number_of_images, dimensions, size, size)，而输出形式是(number_of_images, ouput_channle, size, size)。

In [12]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import torchvision
import torch.utils.data as Data
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

epoch = 1
lr = 0.001
batch_size = 10

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.out = nn.Linear(in_features=32*7*7, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output
    
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


# 数据准备

从torchvison.dataset直接加载MNIST数据集，已经提前下载好，所以从本地提取。验证数据集只采用其中的前2000个。

In [14]:
train_data = torchvision.datasets.MNIST(
        root='D:/PycharmProjects/Pytorch_learn/',
        train=True,
        transform=torchvision.transforms.ToTensor(),
        download=False
    )
train_loader = Data.DataLoader(dataset=train_data,
                                   batch_size=batch_size,
                                   shuffle=True)
test_data = torchvision.datasets.MNIST(root='D:/PycharmProjects/Pytorch_learn/', 
                                      train=False)
test_data_features = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensorb)[:2000]
test_data_labels = test_data.targets[:2000]

# 训练过程

由于采用cpu训练，所以本次只训练一个epoch。已经提前训练好并保存在本地，这里提供训练过程，但不运行

In [15]:
def train_program():
    train_data = prepare_train_data()
    train_loader = Data.DataLoader(dataset=train_data,
                                   batch_size=batch_size,
                                   shuffle=True)

    cnn = CNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(cnn.parameters(), lr=lr)

    for step, (image_tensor, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        output = cnn(image_tensor)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    return cnn

# 加载模型

In [16]:
cnn.load_state_dict(torch.load('D:/PycharmProjects/Pytorch_learn/params.pkl'))
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


# 测试

In [18]:
predictions = torch.max(f.softmax(cnn(test_data_features), dim=1), dim=1)[1]
print(sum(predictions == test_data_labels).item()/2000)

0.8875
